### Customer retention Analytics

##### Business Problem:
An International bank collected a sample data of 10,000 customers. They observed that some of their custimers are leaving or churning in an unusually high rate and they want to find understand and access why their customers keep leaving. They have hiered you as a data scientist to look into the data to give then some insight.

##### Data:
The data was collected within the last 5 months, the feautures include the customer's name, creditscore, geography, gendey,age,tenure,balnce, number of products(accounts),credit card status(whether they have one or not), estimated salary, activity status(active member or not) and if the customer remained with them.

#### Goal:
Your goal is to create a geodemographic segmentation  to identify which of the customers have the highest risk of leaving.

In [1]:
#importing important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
torch.__version__

'1.7.1'

### Working on a banking dataset to detect churn activities

In [3]:
#importing the data
data = pd.read_csv("https://raw.githubusercontent.com/AdeloreSimiloluwa/Artificial-Neural-Network/master/data/Churn_Modelling.csv")

In [ ]:
#viewing the data
data.head()

In [ ]:
#frequency of the target classes
sns.countplot(x='Exited', data=data)

From the data we have more information on the customers that stayed at the bank

In [ ]:
#checking for datatype of each column
data.dtypes

In [ ]:
#checking for missing values
data.isnull().sum()

In [ ]:
#dropping some columns that are not needed
data = data.drop(columns=['RowNumber','CustomerId','Surname'], axis=1)
data.head()

In [ ]:
#viewing the unique values in Geography column
data['Geography'].unique()

In [ ]:
#data features
X = data.iloc[:,:-1]
X.head()

In [ ]:
#target data
y = data.iloc[:,-1:]
y.head()

In [ ]:
#encoding the categorical columns
le = LabelEncoder()
ohe = OneHotEncoder()
X['Gender'] = le.fit_transform(X['Gender'])
geo_df = pd.DataFrame(ohe.fit_transform(X[['Geography']]).toarray())

#getting feature name after onehotencoding
geo_df.columns = ohe.get_feature_names(['Geography'])

#merging geo_df with the main data
X = X.join(geo_df)
X.head()

In [ ]:
#dropping the old columns after encoding
X.drop(columns=['Geography'], axis=1, inplace=True)
X.head()

### Split Data

In [ ]:
X_train,X_test,y_train,y_test = train_test_split( X,y, test_size=0.2, random_state = 42)

In [ ]:
data_dict = {"X_train":X_train, "X_test":X_test, "y_train":y_train, "y_test":y_test}
for i in data_dict:
    print("The shape of {} is {}".format(i,data_dict[i].shape))

### Feature scaling

In [ ]:
sc =StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Deep learning model

In [ ]:
#setting model hyper-parameters
EPOCHS = 1
BATCH_SIZE =10
LEARNING_RATE = 0.001

In [ ]:
X_test.shape

#### Defining custom dataset

In [18]:
#train data
class trainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self,index):
        return self.X_data[index], self.y_data[index]
    
    def __len__(self):
        return len(self.X_data)

train_data = trainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train.values))

In [19]:
#test data
class testData(Dataset):
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self,index):
        return self.X_data[index]
    
    def __len__(self):
        return len(self.X_data)

test_data = testData(torch.FloatTensor(X_test))

### Defining dataloader to read dataset class in batches

In [20]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(dataset=test_data, batch_size=1)

### defining neural network architecture

In [21]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        #number of input features is 12
        self.layer_1 = nn.Linear(12, 16)
        self.layer_2 = nn.Linear(16, 8)
        self.layer_out = nn.Linear(8, 1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(16)
        self.batchnorm2 = nn.BatchNorm1d(8)
        
    #feed forward network
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)

        return x

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [23]:
#initializing optimizer and loss
classifier = binaryClassification()
classifier.to(device)
print(classifier)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr = LEARNING_RATE)

binaryClassification(
  (layer_1): Linear(in_features=12, out_features=16, bias=True)
  (layer_2): Linear(in_features=16, out_features=8, bias=True)
  (layer_out): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [24]:
#function to calculate accuracy
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    
    results_sum = (y_pred_tag == y_test).sum().float()
    acc = results_sum/y_test.shape[0]
    acc =torch.round(acc*100)
    return acc

In [25]:
#training the model
classifier.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        #setting gradient to 0 per mini-batch
        optimizer.zero_grad()
        y_pred = classifier(X_batch)
        loss =criterion(y_pred, y_batch)
        acc = binary_acc(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        print(f'Epoch {e+0:03}: | Loss:{epoch_loss/len(train_loader):.5f} | Acc:{epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss:0.00092 | Acc:0.062
Epoch 001: | Loss:0.00177 | Acc:0.138
Epoch 001: | Loss:0.00266 | Acc:0.212
Epoch 001: | Loss:0.00358 | Acc:0.263
Epoch 001: | Loss:0.00459 | Acc:0.300
Epoch 001: | Loss:0.00545 | Acc:0.375
Epoch 001: | Loss:0.00633 | Acc:0.425
Epoch 001: | Loss:0.00720 | Acc:0.500
Epoch 001: | Loss:0.00815 | Acc:0.537
Epoch 001: | Loss:0.00907 | Acc:0.588
Epoch 001: | Loss:0.00992 | Acc:0.675
Epoch 001: | Loss:0.01081 | Acc:0.750
Epoch 001: | Loss:0.01171 | Acc:0.812
Epoch 001: | Loss:0.01257 | Acc:0.887
Epoch 001: | Loss:0.01339 | Acc:0.975
Epoch 001: | Loss:0.01442 | Acc:1.012
Epoch 001: | Loss:0.01533 | Acc:1.050
Epoch 001: | Loss:0.01626 | Acc:1.087
Epoch 001: | Loss:0.01720 | Acc:1.137
Epoch 001: | Loss:0.01805 | Acc:1.212
Epoch 001: | Loss:0.01898 | Acc:1.262
Epoch 001: | Loss:0.01984 | Acc:1.350
Epoch 001: | Loss:0.02083 | Acc:1.387
Epoch 001: | Loss:0.02161 | Acc:1.462
Epoch 001: | Loss:0.02261 | Acc:1.500
Epoch 001: | Loss:0.02346 | Acc:1.587
Epoch 001: |

Epoch 001: | Loss:0.30894 | Acc:37.275
Epoch 001: | Loss:0.30959 | Acc:37.375
Epoch 001: | Loss:0.31043 | Acc:37.450
Epoch 001: | Loss:0.31113 | Acc:37.550
Epoch 001: | Loss:0.31163 | Acc:37.650
Epoch 001: | Loss:0.31257 | Acc:37.725
Epoch 001: | Loss:0.31321 | Acc:37.837
Epoch 001: | Loss:0.31387 | Acc:37.925
Epoch 001: | Loss:0.31436 | Acc:38.025
Epoch 001: | Loss:0.31504 | Acc:38.100
Epoch 001: | Loss:0.31568 | Acc:38.200
Epoch 001: | Loss:0.31641 | Acc:38.300
Epoch 001: | Loss:0.31726 | Acc:38.388
Epoch 001: | Loss:0.31825 | Acc:38.462
Epoch 001: | Loss:0.31868 | Acc:38.575
Epoch 001: | Loss:0.31933 | Acc:38.675
Epoch 001: | Loss:0.31982 | Acc:38.788
Epoch 001: | Loss:0.32026 | Acc:38.900
Epoch 001: | Loss:0.32105 | Acc:38.987
Epoch 001: | Loss:0.32154 | Acc:39.100
Epoch 001: | Loss:0.32222 | Acc:39.200
Epoch 001: | Loss:0.32306 | Acc:39.288
Epoch 001: | Loss:0.32364 | Acc:39.400
Epoch 001: | Loss:0.32413 | Acc:39.513
Epoch 001: | Loss:0.32497 | Acc:39.587
Epoch 001: | Loss:0.32555

In [26]:
#test model
y_pred_list = []
classifier.eval()
count = 0
#ensures no back propagation during testing and reduces memeory usage
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = classifier(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        
        y_pred_list.append(y_pred_tag.cpu().numpy())
    y_pred_list = [i.squeeze().tolist() for i in y_pred_list] 
    y_pred_list = [bool(i) for i in y_pred_list]

In [27]:
df1 = pd.DataFrame(y_test)
df1.reset_index(inplace=True)
df1.drop(columns=['index'], axis=1, inplace=True)
df2 = pd.DataFrame(y_pred_list)
df = pd.concat([df1, df2], axis=1)
df.columns=['target','predicted']
df.head()

,target,predicted
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False


In [29]:
#confusion matrix
cm = confusion_matrix(df['target'],df['predicted'])
print(cm)

[[1550   57]
 [ 289  104]]


From our confusion matrix we conclude that:
1. **True positive:** 91(We predicted a positive result and it was positive)- the model rightly predicted the ones who left the bank 
2. **True negative:** 1570(We predicted a negative result and it was negative)-the model rightly predicted the ones who stayed at the bank 
3. **False positive:** 37(We predicted a positive result and it was negative)-the model predicted that these ones left when they actually stayed
4. **False negative:** 302(We predicted a negative result and it was positive)- the model predicted that these ones stayed when they actually left

In [48]:
#classification report
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.84      0.98      0.90      1607
           1       0.71      0.23      0.35       393

   micro avg       0.83      0.83      0.83      2000
   macro avg       0.77      0.60      0.63      2000
weighted avg       0.81      0.83      0.79      2000

